In [ ]:
#sudo apt-get install -y xvfb libnss3 libgconf-2-4
#wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
#sudo apt install ./google-chrome-stable_current_amd64.deb



from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import csv
import time

# Configura opciones para ejecutar Chrome de manera gráfica
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# Inicializa el driver de Chrome
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

def extract_data_from_page():
    WebDriverWait(driver, 20).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".rf-p"))
    )
    
    containers = driver.find_elements(By.CSS_SELECTOR, ".rf-p")
    data = []

    for container in containers:
        try:
            header = container.find_element(By.CSS_SELECTOR, ".rf-p-hdr")
            casacion = header.find_element(By.XPATH, ".//td[2]/span").text.strip()
            nroexp = header.find_element(By.XPATH, ".//td[3]/span").text.strip()

            body = container.find_element(By.CSS_SELECTOR, ".rf-p-b")
            pretension = body.find_element(By.XPATH, ".//div[contains(text(), 'Pretensión/Delito:')]/following-sibling::div").text.strip()
            tipo_resolucion = body.find_element(By.XPATH, ".//div[contains(text(), 'Tipo Resolución:')]/following-sibling::div").text.strip()
            fecha_resolucion = body.find_element(By.XPATH, ".//div[contains(text(), 'Fecha Resolución:')]/following-sibling::div").text.strip()
            sala_suprema = body.find_element(By.XPATH, ".//div[contains(text(), 'Sala Suprema:')]/following-sibling::div").text.strip()
            norma_derecho_interno = body.find_element(By.XPATH, ".//div[contains(text(), 'Norma de Derecho Interno:')]/following-sibling::div").text.strip()
            sumilla = body.find_element(By.XPATH, ".//div[contains(text(), 'Sumilla:')]/following-sibling::div").text.strip()
            palabras_clave = body.find_element(By.XPATH, ".//div[contains(text(), 'Palabras Clave:')]/following-sibling::div").text.strip()

            # Extraer enlace del botón "ver resolución"
            enlace_resolucion = container.find_element(By.XPATH, ".//td/a[contains(@href, 'ServletDescarga')]").get_attribute('href')
            
            data.append([
                casacion, nroexp, pretension, tipo_resolucion, fecha_resolucion,
                sala_suprema, norma_derecho_interno, sumilla, palabras_clave, enlace_resolucion
            ])
        except Exception as e:
            print(f"Error al procesar el contenedor: {e}")

    return data

try:
    driver.get("https://jurisprudencia.pj.gob.pe/jurisprudenciaweb/faces/page/inicio.xhtml")
    print("Página principal cargada.")

    WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.ID, "formBuscador:especializada:header:inactive"))
    ).click()
    print("Filtro especializado activado.")

    
    select_especialidad = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.ID, "formBuscador:buEspecialidad"))
    )

    #seleccionar el  VALUE para cada especialidad

# Lista de grupos con sus respectivos registros, ordenados por ID
# registros = [
#     {"id": 1, "categoria": "Civil", "registros": 1330, "estado": "ya escraperado"},------------------listo-----------------
#     {"id": 2, "categoria": "Comercial", "registros": 50, "estado": "ya escraperado"},------------------listo-----------------
#     {"id": 3, "categoria": "Contencioso Administrativo", "registros": 2189},
#     {"id": 4, "categoria": "Familia Civil", "registros": 168},
#     {"id": 5, "categoria": "Familia Tutelar", "registros": 17, "estado": "ya escraperado"},------------------listo-----------------
#     {"id": 6, "categoria": "Familia Penal", "registros": 23, "estado": "ya escraperado"},------------------listo-----------------
#     {"id": 7, "categoria": "Contencioso Adm Laboral", "registros": 1851},
#     {"id": 8, "categoria": "Contencioso Adm Previsional", "registros": 1266},
#     {"id": 9, "categoria": "Laboral", "registros": 2341},
#     {"id": 10, "categoria": "Penal", "registros": 6212},
#     {"id": 11, "categoria": "Revisión de Procedimiento Coactivo", "registros": 1204},
#     {"id": 12, "categoria": "Constitucional", "registros": 688}
# ]

#.................................................................|...........
#.................................................................|...........
#.................................................................|...........
#.................................................................|...........
#.................................................................|...........
#.................................................................v...........    
    select_especialidad.find_element(By.XPATH, ".//option[@value='6']").click()
    print("Opción 'Civil' seleccionada.")

    WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.NAME, "formBuscador:j_idt69"))
    ).click()
    print("Botón de búsqueda clicado.")

    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, ".rf-p"))
    )
    print("Resultados de búsqueda cargados.")
##########-------CAMBIAR NOMBRE DE CARPETA SEGUN CATEGORIA--------###########
#......................|......................................................
#......................|......................................................
#......................|......................................................
#......................|......................................................
#......................|......................................................
#......................v......................................................  

    with open('familia-penal.csv', mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow([
            "Casación", "Número de Expediente", "Pretensión/Delito", "Tipo Resolución",
            "Fecha Resolución", "Sala Suprema", "Norma de Derecho Interno", "Sumilla", "Palabras Clave", "Enlace Resolución"
        ])

##########-------CAMBIAR NOMBRE DE PAGINAS SEGUN CATEGORIA--------###########
#............................|......................................................
#............................|......................................................
#............................|......................................................
#............................|......................................................
#............................|......................................................
#............................v......................................................  
        for page in range(1, 100000):  # Cambiado para las  200 páginas
            retries = 3  # Intentar reintentar la carga de la página 3 veces
            for attempt in range(retries):
                try:
                    print(f"Extrayendo datos de la página {page}, intento {attempt + 1}/{retries}")
                    data = extract_data_from_page()
                    writer.writerows(data)
                    print(f"Datos extraídos de la página {page}")
                    break
                except Exception as e:
                    print(f"Error al extraer datos de la página {page}, reintento {attempt + 1}/{retries}")
                    time.sleep(2)  # Esperar un momento antes de reintentar



#modificar el umbral para delimitar el numero de paginas a obtener segun la categoria


##########-------CAMBIAR NOMBRE DE PAGINAS SEGUN CATEGORIA--------###########
#......................|......................................................
#......................|......................................................
#......................|......................................................
#......................|......................................................
#......................|......................................................
#......................v......................................................  
            if page < 100000:
                try:
                    next_page_button = driver.find_element(By.ID, f"formBuscador:data1_ds_{page + 1}")
                    WebDriverWait(driver, 20).until(
                        EC.element_to_be_clickable(next_page_button)
                    ).click()
                    print(f"Navegando a la página {page + 1}")

                    WebDriverWait(driver, 20).until(
                        EC.invisibility_of_element((By.ID, "panelState_shade"))
                    )
                    time.sleep(5)  # Esperar un momento adicional para asegurarse de que la página se ha cargado
                    WebDriverWait(driver, 20).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, ".rf-p"))
                    )
                except Exception as e:
                    print(f"Error al hacer clic en el botón de la siguiente página: {e}")
                    break  # Rompe el bucle si ocurre un error

finally:
    driver.quit()
    print("Proceso completado con.", page, "paginas revisadas")


Página principal cargada.
Filtro especializado activado.
Opción 'Civil' seleccionada.
Botón de búsqueda clicado.
Resultados de búsqueda cargados.
Extrayendo datos de la página 1, intento 1/3
Error al procesar el contenedor: Message: no such element: Unable to locate element: {"method":"css selector","selector":".rf-p-hdr"}
  (Session info: chrome=133.0.6943.141); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
#0 0x62d695bf714a <unknown>
#1 0x62d695694b80 <unknown>
#2 0x62d6956e60e9 <unknown>
#3 0x62d6956e6271 <unknown>
#4 0x62d6956d9b36 <unknown>
#5 0x62d69570befd <unknown>
#6 0x62d6956d9a2a <unknown>
#7 0x62d69570c09e <unknown>
#8 0x62d69573214a <unknown>
#9 0x62d69570bca3 <unknown>
#10 0x62d6956d7f08 <unknown>
#11 0x62d6956d9071 <unknown>
#12 0x62d695bc0b5b <unknown>
#13 0x62d695bc4ae2 <unknown>
#14 0x62d695bac967 <unknown>
#15 0x62d695bc56d4 <unknown>
#16 0x62d695b90c7f <unk

In [ ]:
import csv
import requests
import os

# Nombre del archivo CSV  (cambiar para cada categoria)
csv_file = 'familia_tutelar.csv'

# Nombre de la carpeta donde se guardarán los PDFs (cambiar para cada categoria)
output_folder = 'pdf_files'

# Crear la carpeta si no existe
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Leer el archivo CSV y descargar los PDFs
with open(csv_file, mode='r', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    for row in reader:
        pdf_url = row['Enlace Resolución']
        pdf_name = row['Número de Expediente'] + '.pdf'
        pdf_path = os.path.join(output_folder, pdf_name)
        
        try:
            response = requests.get(pdf_url)
            response.raise_for_status()
            with open(pdf_path, 'wb') as pdf_file:
                pdf_file.write(response.content)
            print(f"Descargado: {pdf_name}")
        except requests.exceptions.RequestException as e:
            print(f"Error al descargar {pdf_name}: {e}")

Descargado: 004407-2024.pdf
Descargado: 035218-2022.pdf
Descargado: 018932-2021.pdf
Descargado: 007940-2014.pdf
Descargado: 000412-2020.pdf
Descargado: 006414-2019.pdf
Descargado: 000864-2022.pdf
Descargado: 011995-2021.pdf
Descargado: 024491-2021.pdf
Descargado: 006020-2018.pdf
Descargado: 021619-2022.pdf
Descargado: 000886-2018.pdf
Descargado: 001981-2021.pdf
Descargado: 037311-2022.pdf
Descargado: 046159-2022.pdf
Descargado: 018133-2019.pdf
Descargado: 003613-2021.pdf
Descargado: 002131-2022.pdf
Descargado: 011081-2020.pdf
Descargado: 003736-2019.pdf
Descargado: 003606-2019.pdf
Descargado: 002300-2021.pdf
Descargado: 002304-2021.pdf
Descargado: 030778-2022.pdf
Descargado: 030491-2022.pdf
Descargado: 001048-2020.pdf
Descargado: 000812-2020.pdf
Descargado: 003438-2019.pdf
Descargado: 001026-2020.pdf
Descargado: 044801-2022.pdf
Descargado: 019243-2021.pdf
Descargado: 002446-2021.pdf
Descargado: 001524-2021.pdf
Descargado: 002742-2021.pdf
Descargado: 001472-2021.pdf
Descargado: 002808-2